In [1]:
import os
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
import json
from langchain_core.messages import ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from tools.get_function import GetFuncTool

os.environ["OPENAI_API_KEY"] = "sk-579198d69b204050925290d4782091ea"
os.environ["TAVILY_API_KEY"] = "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"
# os.environ["DEEPSEEK_API_KEY"] = "sk-H4yqO8alBHpXtYxj92790f0fFa3b45C5Be57F7A37d703b6b"
# os.environ["DEEPSEEK_API_BASE"] = "https://www.gptapi.us/v1"

# tool = TavilySearchResults(max_results=2)
# tools = [tool]
# print(tool.invoke("What's a 'node' in LangGraph?"))
tool = GetFuncTool(target="mihome")
tools = [tool]
llm = ChatOpenAI(model="qwen-plus", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
llm_with_tools = llm.bind_tools(tools)


In [3]:

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

def route_tools(
    state: State,
):
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END

tool_node = ToolNode(tools=[tool])
graph_builder = StateGraph(State)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    {"tools": "tools", END: END},
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()


# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph().draw_mermaid_png()))
# except Exception:
#     # This requires some extra dependencies and is optional
#     pass


In [4]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)
            

user_input = "com.xiaomi.smarthome.framework.plugin.rn.nativemodule.MIOTPersistModuleCore#copyFile主要做了什么"
print("User: " + user_input)
stream_graph_updates(user_input)



User: com.xiaomi.smarthome.framework.plugin.rn.nativemodule.MIOTPersistModuleCore#copyFile主要做了什么
Assistant: 
Assistant: import android.content.Context;
import android.text.TextUtils;
import com.facebook.react.bridge.Callback;
import com.facebook.react.bridge.ReactMethod;
import com.facebook.react.bridge.ReadableMap;
import com.google.android.exoplayer2.C;
import com.xiaomi.onetrack.b.e;
import com.xiaomi.smarthome.core.entity.Error;
import com.xiaomi.smarthome.framework.plugin.rn.utils.MediaStoreUtil;
import com.xiaomi.smarthome.framework.plugin.rn.utils.RnCallbackMapUtil;
import com.xiaomi.smarthome.framework.plugin.rn.utils.RnReadableMapUtil;
import java.io.File;
import java.util.Map;
import rx.Observable;
import rx.functions.Action1;
import rx.schedulers.Schedulers;
@ReactMethod
public void copyFile(ReadableMap params, final Callback callback) {
    final String srcPath = RnReadableMapUtil.optString(params, "srcPath");
    final String dstPath = RnReadableMapUtil.optString(params, "